In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Merge, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
np.random.seed(12345)

In [3]:
mapping_dict = {
'Gender': {'male':0, 'female':1, 'no_gender':2},
}


In [4]:
## load data
products = pd.read_csv('products.csv')
tender = pd.read_csv('tenderModes.csv')
sample = pd.read_csv('sampleSubmission.csv')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
products['promotion_description'].fillna('no_promo', inplace=True)
products['Gender'].fillna('no_gender', inplace=True)
products['State'].fillna('no_state', inplace=True)
products['PinCode'].fillna(-1, inplace=True)
products['DOB'].fillna("1", inplace=True)

In [6]:
#handling missing data for product_code
products = products[np.isfinite(products['product_code'])]

In [7]:
products_new = products[products['customerID'].isin(sample['customerID'])]

In [8]:
products_new['product_code'] = products_new['product_code'].astype('int64')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [9]:
cnt_srs = products['product_code'].astype('int64').value_counts().reset_index().head(20)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [14]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)
    
product_joined = pd.merge(products_new, cnt_srs, on='product_code', how='inner')

products_less = product_joined[['customerID', 'Gender', 'State', 'product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)

products_to_join = pd.DataFrame(products_new['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

# Join the dummy variables to the main dataframe
products_final = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
products_final.head()

,customerID,Gender,State,product_code
0,BBID_20413,no_gender,MADHYA PRADESH,300840018.0
1,BBID_20413,no_gender,MADHYA PRADESH,300840018.0
2,BBID_20413,no_gender,MADHYA PRADESH,300840018.0
3,BBID_20413,no_gender,MADHYA PRADESH,300840018.0
4,BBID_20413,no_gender,MADHYA PRADESH,300840018.0


In [15]:
products_final.dropna(inplace=True)

prod_dict = {'_'}
products_final = pd.get_dummies(products_final, columns=['product_code'], prefix=prod_dict)
products_final.columns=products_final.columns.str.replace('_','')
products_final.reset_index(drop=True).head()

,customerID,Gender,State,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
0,BBID_20413,no_gender,MADHYA PRADESH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,BBID_20413,no_gender,MADHYA PRADESH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,BBID_20413,no_gender,MADHYA PRADESH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,BBID_20413,no_gender,MADHYA PRADESH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,BBID_20413,no_gender,MADHYA PRADESH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
products_final.drop_duplicates(subset=['customerID'], keep='last', inplace=True)
products_final.reset_index(drop=True).head()

,customerID,Gender,State,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
0,BBID_20415,male,MADHYA PRADESH,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_20431,female,no_state,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,BBID_20436,male,no_state,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,BBID_20437,female,MADHYA PRADESH,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_20442,no_gender,MADHYA PRADESH,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
## convert data into numeric / float
from sklearn.preprocessing import LabelEncoder, RobustScaler

for c in products_final.columns:
    if c not in ['customerID', 'product_code']:
        lbl = LabelEncoder()
        products_final[c] = products_final[c].factorize()[0]

    if products_final[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        products_final[c] = lbl.fit_transform(products[c])

In [21]:
products_final.head()

,customerID,Gender,State,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
203,BBID_20415,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
289,BBID_20431,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
298,BBID_20436,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
412,BBID_20437,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
420,BBID_20442,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
products_final.reset_index(drop=True).head()

,customerID,Gender,State,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
0,BBID_20415,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_20431,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,BBID_20436,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,BBID_20437,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_20442,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
